In [1]:
import sqlalchemy

print(sqlalchemy.__version__)

1.4.45


# Select functions

In [2]:
from sqlalchemy import create_engine, MetaData, Table, select, func

In [3]:
engine = create_engine("sqlite:///data/census.sqlite")

In [4]:
# Criando uma conexão com o banco para executar comandos
connection = engine.connect()

In [5]:
metadata = MetaData()
census = Table('census', metadata, autoload=True, autoload_with=engine)

In [7]:
# Obtendo os valores distintos da coluna state e depois contando esses valores
statement = select([
    func.count(
        func.distinct(census.columns.state)
    )
])

# Quando o resultado é apenas 1 valor nós podemos usar .scalar(), ao invés de .fetchall() ou outro, para retornar apenas o valor
distinct_state_count = connection.execute(statement).scalar()
distinct_state_count

51

In [12]:
# Selecionando a coluna state e uma contagem dos valores de age
statement = select([census.columns.state, func.count(census.columns.age)])

# Agrupando o statement anterior por valor de state, assim nós contamos a quantidade de valores de age em cada estado
statement = statement.group_by(census.columns.state)

print(statement)
connection.execute(statement).fetchmany(5)

SELECT census.state, count(census.age) AS count_1 
FROM census GROUP BY census.state


[('Alabama', 344),
 ('Alaska', 344),
 ('Arizona', 344),
 ('Arkansas', 344),
 ('California', 344)]

In [20]:
# Criando uma coluna chamada population que terá a soma dos valores de pop2008
pop2008_sum = func.sum(census.columns.pop2008).label('population')

# Selecionando a coluna state e a coluna population
statement = select([census.columns.state, pop2008_sum])

# Agrupando population por state, então cada linha terá o nome de um estado e sua população em 2008
statement = statement.group_by(census.columns.state)

print(statement)
connection.execute(statement).fetchmany(5)

SELECT census.state, sum(census.pop2008) AS population 
FROM census GROUP BY census.state


[('Alabama', 9298734),
 ('Alaska', 1329092),
 ('Arizona', 12961534),
 ('Arkansas', 5696864),
 ('California', 73218004)]